In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time
from spikingjelly.activation_based import neuron, functional, layer

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Current device: {torch.cuda.get_device_name(0)}")
    print(f"Device count: {torch.cuda.device_count()}") # 应该输出 1

PyTorch version: 2.9.1+cu128
CUDA available: True
Current device: NVIDIA GeForce RTX 3090
Device count: 1


In [2]:
import psutil
import os

# 获取当前进程内存使用
process = psutil.Process(os.getpid())
memory_info = process.memory_info()
print(f"当前Python进程内存使用: {memory_info.rss / 1024 / 1024:.2f} MB")

# 获取系统总内存信息
memory = psutil.virtual_memory()
print(f"总内存: {memory.total / 1024 / 1024 / 1024:.2f} GB")
print(f"已使用: {memory.used / 1024 / 1024 / 1024:.2f} GB")
print(f"可用内存: {memory.available / 1024 / 1024 / 1024:.2f} GB")
print(f"内存使用率: {memory.percent}%")

当前Python进程内存使用: 672.29 MB
总内存: 251.56 GB
已使用: 7.05 GB
可用内存: 242.70 GB
内存使用率: 3.5%


In [3]:
from spikingjelly.datasets.cifar10_dvs import CIFAR10DVS
import matplotlib.pyplot as plt
# 数据下载路径
root = './CIFAR10DVS'

In [ ]:
# 选择 data_type: 'event', 'frame', 'voxel'
import os
os.makedirs('CIFAR10DVS', exist_ok=True)
dataset_event  = CIFAR10DVS(root, data_type='event')

# 注意：pip 上的 spikingjelly 0.0.0.0.14 CIFAR10-DVS 转换代码多线程解析会 silent fail
# 后续手动将aedat文件转化为npz文件

The [./CIFAR10DVS/download] directory for saving downloaded files already exists, check files...
The directory [./CIFAR10DVS/extract] for saving extracted files already exists.
SpikingJelly will not check the data integrity of extracted files.
If extracted files are not integrated, please delete [./CIFAR10DVS/extract] manually, then SpikingJelly will re-extract files from [./CIFAR10DVS/download].
Mkdir [./CIFAR10DVS/events_np].
Start to convert the origin data from [./CIFAR10DVS/extract] to [./CIFAR10DVS/events_np] in np.ndarray format.
Mkdir [./CIFAR10DVS/events_np/deer].
Start to convert [./CIFAR10DVS/extract/deer/cifar10_deer_157.aedat] to [./CIFAR10DVS/events_np/deer/cifar10_deer_157.npz].
Start to convert [./CIFAR10DVS/extract/deer/cifar10_deer_995.aedat] to [./CIFAR10DVS/events_np/deer/cifar10_deer_995.npz].
Start to convert [./CIFAR10DVS/extract/deer/cifar10_deer_545.aedat] to [./CIFAR10DVS/events_np/deer/cifar10_deer_545.npz].
Start to convert [./CIFAR10DVS/extract/deer/cifar10

FileNotFoundError: Found no valid file for the classes airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck. Supported extensions are: .npz

In [9]:
aedat_file = './CIFAR10DVS/extract/deer/cifar10_deer_157.aedat'

with open(aedat_file, 'rb') as f:
    head = f.read(200)  # 读取前 200 字节
    print(head.decode(errors='ignore'))

print(aedat_file, os.path.getsize(aedat_file))

#!AER-DAT2.0
# This is a raw AE data file - do not edit
# Data format is int32 address, int32 timestamp (8 bytes total), repeated for each event
# Timestamps tick is 1 us
# created Tue Apr 29 11:3
./CIFAR10DVS/extract/deer/cifar10_deer_157.aedat 1079296


In [7]:
# 手动将aedat文件转化为npz文件
import os
import numpy as np
from glob import glob

def convert_aedat2_to_npz_fixed(aedat_path, npz_path):
    # 创建目标目录
    os.makedirs(os.path.dirname(npz_path), exist_ok=True)

    with open(aedat_path, 'rb') as f:
        content = f.read()

    # 按行分割，跳过 header 注释行
    lines = content.split(b'\n')
    idx = 0
    for i, line in enumerate(lines):
        if not line.startswith(b'#') and len(line.strip()) > 0:
            idx = i
            break
    data = b'\n'.join(lines[idx:])

    # 确保长度是 8 的倍数
    extra = len(data) % 8
    if extra != 0:
        data = data[:-extra]

    # big-endian int32
    events = np.frombuffer(data, dtype='>i4').reshape(-1,2)
    addr = events[:,0]
    t = events[:,1].astype(np.int64)

    # 解析 x/y/p
    x = addr & 0x7f
    y = (addr >> 7) & 0x7f
    p = (addr >> 14) & 1

    np.savez(npz_path, x=x, y=y, t=t, p=p)
    print(f"Saved {npz_path}")

root_extract = './CIFAR10DVS/extract'
root_npz = './CIFAR10DVS/events_np'

classes = sorted(os.listdir(root_extract))
for c in classes:
    class_path = os.path.join(root_extract, c)
    files = glob(os.path.join(class_path, '*.aedat'))
    for f in files:
        fname = os.path.basename(f)
        npz_file = os.path.join(root_npz, c, fname.replace('.aedat', '.npz'))
        convert_aedat2_to_npz_fixed(f, npz_file)

Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_311.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_285.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_918.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_477.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_777.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_497.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_470.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_538.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_41.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_304.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_193.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_830.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_373.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_422.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_793.npz
Saved ./CIFAR10DVS/events_np/airplane/cifar10_airplane_9

In [1]:
# 读取aedat文件，展示内容
import numpy as np

def read_cifar10dvs_aedat(file_name):
    with open(file_name, 'rb') as f:
        data = f.read()

    # 跳过 header
    lines = data.split(b'\n')
    idx = 0
    for i, line in enumerate(lines):
        if not line.startswith(b'#') and len(line.strip()) > 0:
            idx = i
            break

    event_data = b'\n'.join(lines[idx:])

    # 确保 8 的倍数
    extra = len(event_data) % 8
    if extra != 0:
        event_data = event_data[:-extra]

    # big-endian int32
    events = np.frombuffer(event_data, dtype='>i4').reshape(-1,2)
    addresses = events[:,0]
    timestamps = events[:,1]

    x = addresses & 0x7f
    y = (addresses >> 7) & 0x7f
    p = (addresses >> 14) & 1
    t = timestamps

    return {'x': x, 'y': y, 'p': p, 't': t}

f = './CIFAR10DVS/extract/deer/cifar10_deer_876.aedat'
events = read_cifar10dvs_aedat(f)

print(events['x'][:10])
print(events['y'][:10])
print(events['p'][:10])
print(events['t'][:10])

[ 41  81  39  53  65 103 125  15  19 127]
[113  80  99  63   3 126   4  45 101 102]
[1 1 1 1 1 0 0 0 0 0]
[0 1 1 2 2 2 3 3 4 4]


In [2]:
import os
import numpy as np

events_np_dir = './CIFAR10DVS/events_np/deer'  # 例如 deer 类
npz_files = [f for f in os.listdir(events_np_dir) if f.endswith('.npz')]
print("找到的前五个 npz 文件:", npz_files[:5])  # 显示前 5 个文件
npz_path = os.path.join(events_np_dir, npz_files[0])
data = np.load(npz_path)

print("文件包含的数组：", data.files)  # 通常 ['x', 'y', 'p', 't']

# 打印前 10 个事件
print("x:", data['x'][:10])
print("y:", data['y'][:10])
print("p:", data['p'][:10])
print("t:", data['t'][:10])

找到的前五个 npz 文件: ['cifar10_deer_876.npz', 'cifar10_deer_60.npz', 'cifar10_deer_965.npz', 'cifar10_deer_654.npz', 'cifar10_deer_319.npz']
文件包含的数组： ['x', 'y', 't', 'p']
x: [ 41  81  39  53  65 103 125  15  19 127]
y: [113  80  99  63   3 126   4  45 101 102]
p: [1 1 1 1 1 0 0 0 0 0]
t: [0 1 1 2 2 2 3 3 4 4]
